In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [18]:
default_threshold = 0.10   # eg 0.1 = 10%
file_dir = 'D:/Astro/Images/2024-12-20/Lump Star/LIGHTS'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [19]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))
data = data[data['FilePath'].str.strip() != '' ]
data

,ExposureNumber,FilePath,FilterName,ExposureStart,Duration,Binning,CameraTemp,CameraTargetTemp,Gain,Offset,...,GuidingRMSDECArcSec,FocuserPosition,FocuserTemp,RotatorPosition,PierSide,Airmass,ExposureStartUTC,MountRA,MountDec,FileName
0,0.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 22:17,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,308.0,NaN,NaN,NaN,1.4318,2024-12-20T11:17:22.0560881Z,85.416221,-2.265237,2024-12-20_22-17-22_NoFilter_400_15.00s_0000.cr3
1,1.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 22:17,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,308.0,NaN,NaN,NaN,1.4304,2024-12-20T11:17:39.8832961Z,85.415885,-2.264953,2024-12-20_22-17-39_NoFilter_400_15.00s_0001.cr3
2,2.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 22:17,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,308.0,NaN,NaN,NaN,1.4292,2024-12-20T11:17:57.9490710Z,85.416588,-2.265366,2024-12-20_22-17-57_NoFilter_400_15.00s_0002.cr3
3,3.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 22:18,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,308.0,NaN,NaN,NaN,1.4279,2024-12-20T11:18:15.8539184Z,85.416893,-2.265173,2024-12-20_22-18-15_NoFilter_400_15.00s_0003.cr3
4,4.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 22:18,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,308.0,NaN,NaN,NaN,1.4266,2024-12-20T11:18:33.8743969Z,85.417432,-2.265418,2024-12-20_22-18-33_NoFilter_400_15.00s_0004.cr3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,241.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 23:34,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,316.0,NaN,NaN,NaN,1.2194,2024-12-20T12:34:23.9025531Z,85.494232,-2.266872,2024-12-20_23-34-23_NoFilter_400_15.00s_0241.cr3
242,242.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 23:34,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,316.0,NaN,NaN,NaN,1.2189,2024-12-20T12:34:41.8054383Z,85.494458,-2.266862,2024-12-20_23-34-41_NoFilter_400_15.00s_0242.cr3
243,243.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 23:34,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,316.0,NaN,NaN,NaN,1.2185,2024-12-20T12:34:59.8003789Z,85.494970,-2.266755,2024-12-20_23-34-59_NoFilter_400_15.00s_0243.cr3
244,244.0,D:/Astro/Images/2024-12-20/Lump Star/LIGHTS/20...,NoFilter,20/12/2024 23:35,15.0,1x1,NaN,NaN,400.0,-1.0,...,0.0,316.0,NaN,NaN,NaN,1.2180,2024-12-20T12:35:17.7860802Z,85.495119,-2.266855,2024-12-20_23-35-17_NoFilter_400_15.00s_0244.cr3


In [20]:
def filter_percentiles(df, col='HFR', top=0.05, bottom=default_threshold, zero=True):
    top_threshold = df[col].quantile(1 - top) if top else False
    bottom_threshold = df[col].quantile(bottom) if bottom else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom
    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})')
    return is_filtered

In [21]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', bottom=False)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', bottom=False)
is_badFWHM = filter_percentiles(data, col='FWHM', bottom=False)
is_badEccentricity = filter_percentiles(data, col='Eccentricity')
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', top=False)
is_rejected = is_badFWHM | is_badHFR | is_badFWHM | is_badHFRStDev | is_badEccentricity | is_badDetectedStars
print(f'Total Rejected: n({is_rejected.sum()}/{len(data)})')


**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****
HFR: TopThreshold=2.968 n(13), BottomThreshold=0.000 n(0), na=n(0), zero=n(0), filtered=n(13)
HFRStDev: TopThreshold=0.235 n(13), BottomThreshold=0.000 n(0), na=n(0), zero=n(0), filtered=n(13)
FWHM: TopThreshold=13.675 n(12), BottomThreshold=0.000 n(0), na=n(8), zero=n(0), filtered=n(20)
Eccentricity: TopThreshold=0.681 n(12), BottomThreshold=0.318 n(24), na=n(8), zero=n(0), filtered=n(44)
DetectedStars: TopThreshold=0.000 n(0), BottomThreshold=600.500 n(25), na=n(0), zero=n(0), filtered=n(25)
Total Rejected: n(59/246)


In [22]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [24]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')

Moved: 2024-12-20_22-17-22_NoFilter_400_15.00s_0000.cr3
Moved: 2024-12-20_22-20-57_NoFilter_400_15.00s_0012.cr3
Moved: 2024-12-20_22-22-09_NoFilter_400_15.00s_0016.cr3
Moved: 2024-12-20_22-24-32_NoFilter_400_15.00s_0024.cr3
Moved: 2024-12-20_22-24-50_NoFilter_400_15.00s_0025.cr3
Moved: 2024-12-20_22-25-08_NoFilter_400_15.00s_0026.cr3
Moved: 2024-12-20_22-25-26_NoFilter_400_15.00s_0027.cr3
Moved: 2024-12-20_22-26-02_NoFilter_400_15.00s_0029.cr3
Moved: 2024-12-20_22-26-37_NoFilter_400_15.00s_0031.cr3
Moved: 2024-12-20_22-31-05_NoFilter_400_15.00s_0046.cr3
Moved: 2024-12-20_22-31-23_NoFilter_400_15.00s_0047.cr3
Moved: 2024-12-20_22-31-41_NoFilter_400_15.00s_0048.cr3
Moved: 2024-12-20_22-31-59_NoFilter_400_15.00s_0049.cr3
Moved: 2024-12-20_22-32-35_NoFilter_400_15.00s_0051.cr3
Moved: 2024-12-20_22-33-10_NoFilter_400_15.00s_0053.cr3
Moved: 2024-12-20_22-33-28_NoFilter_400_15.00s_0054.cr3
Moved: 2024-12-20_22-33-46_NoFilter_400_15.00s_0055.cr3
Moved: 2024-12-20_22-34-22_NoFilter_400_15.00s_0